# Segmentació

La segmentació d'imatges és una tècnica que divideix una imatge en diferents regions o segments per facilitar-ne l'anàlisi. Donada una imatge d'entrada, la sortida d'un procés de segmentació és una nova imatge on cada pixel té assignada una classe.

Existeixen diferents tipus de segmentació d'imatges

**Segmentació semàntica**

A la segmentació semàntica, el conjunt de píxels d'una imatge s'identifica i es classifica assignant una etiqueta de classe en funció de les seves característiques com ara el color, la textura i la forma. Això proporciona un mapa de píxels d'una imatge (mapa de segmentació) per permetre una anàlisi de la imatge més detallada i precisa. Típicament es seleccionen una o diverses classes d'objectes que apareixen en el conjunt de dades, tots els píxels que no formen part de classes d'interés s'etiqueten com a "fons".

Per exemple, si es volen segmentar els arbres d'una imatge,  tots els píxels relacionats amb un "arbre" s'etiquetarien amb el mateix nom d'objecte sense distingir entre arbres individuals. Un altre exemple seria que un grup de persones en una imatge seria etiquetat com a objecte únic com a "persones".

**Segmentació d'instàncies** (Instance Segmentation)

La segmentació d'instàncies és més sofisticada, ja que implica identificar i delimitar cada objecte individual dins d'una imatge. Per tant, la segmentació d'instàncies va més enllà d'identificar objectes en una imatge, sinó que també delimita els límits exactes de cada instància individual d'aquest objecte. Seguim amb el concepte de la segmentació semàntica on tots els píxels que no formen part de classes d'interés s'etiqueten com a "fons".

Per tant, aquesta tècnica ha de diferenciar entre objectes separats de la mateixa classe. Per exemple, si hi ha molts moixos en una imatge, la segmentació de la instància identificaria cada moix específic. El mapa de segmentació es crea per a cada píxel individual i s'assignen etiquetes separades a instàncies específiques d'objecte mitjançant la creació d'etiquetes diferents que representaran un "moix" diferent al grup de moixos d'una imatge.


![segmentacio](img/segmentacio_semantica.png "Segmentació")


**Segmentació panòptica**

La segmentació panòptica va un pas més enllà en la segmentació d'imatges, combinant les característiques i els processos de les tècniques de segmentació semàntica i d'instàncies. Així, l'algoritme de segmentació panòptica crea una anàlisi d'imatge completa classificant simultàniament cada píxel i identificant diferents instàncies d'objecte de la mateixa classe.

Així, a partir d'una imatge amb diversos cotxes i vianants en un senyal de trànsit, la segmentació panòptica etiquetaria tots els "vianants" i "cotxes" (segmentació semàntica) i també classificaria els diferents escenaris de l'escena, com ara senyals de trànsit, semàfors i tots els altres edificis o fons. Així, la segmentació panòptica detecta i interpreta tot els elements dins d'una imatge determinada.

![segmentacio](img/panoptic.png "Segmentació")


#### Estat de la qüestió

Existeixen moltíssimes xarxes que permeten fer aquesta tasca de segmentació, nosaltres en veurem 4 diferents durant aquest final de semestre, les hem elegit per diferents motius; Algunes d'elles per la seva extrema utilitat com la UNet o la YOLO. Alguna altra com la Mask-RCNN per la seva importància en la història de la IA i la SAM degut a que representa l'estat de l'art en aquesta tasca.

Documentació [enllaç](https://pytorch.org/vision/main/models.html#semantic-segmentation).

## UNET

U-Net: Convolutional Networks for Biomedical Image Segmentation [article](https://arxiv.org/pdf/1505.04597)  (2015)

U-Net és una xarxa neuronal convolucional (CNN) dissenyada per a tasques de segmentació semàntica. El seu nom deriva de la seva arquitectura en forma d'U, que consisteix en codificador seguit d'un camí  descodificador. Aquesta estructura única permet a U-Net capturar el context a diferents escales mantenint la informació espacial.

A la introducció de l'article s'explica:

>En aquest article, ens basem en una arquitectura elegant, l'anomenada "xarxa totalment convolucional". **Modifiquem i ampliem aquesta arquitectura (la convolucional) de manera que funcioni amb molt poques imatges d'entrenament i produeixi segmentacions precises**. La idea principal  és complementar una xarxa convolucional habitual amb successives capes, on els operadors de agrupació (_max pooling_) són substituïts per operadors de mostreig superior (_up sampling_) que augmenten la resolució de la sortida. Per tal de localitzar les característiques d'alta resolució, es combinen característiques del codificador amb les de decodificació.
Una modificació important a la nostra arquitectura és que a la part de mostreig superior també tenim un gran nombre de canals de característiques, que permeten a la xarxa propagar informació de context a capes de resolució més alta. Com a conseqüència, el camí expansiu és més o menys simètric al camí de de codificació i dóna una arquitectura en forma d'U. La xarxa no té capes completament connectades i només utilitza la part vàlida de cada convolució, és a dir, el mapa de segmentació només conté els píxels, per als quals el context complet està disponible a la imatge d'entrada.
Aquesta estratègia permet la segmentació perfecta d'imatges arbitràriament grans mitjançant una estratègia de solapament de regions d'imatges. Per predir els píxels a la regió de la vora de la imatge, el context que falta s'extrapola reflectint la imatge d'entrada. Aquesta estratègia de mosaic és important per aplicar la xarxa a imatges grans, ja que, en cas contrari, la resolució estaria limitada per la memòria de la GPU.

Aquesta xarxa funciona bé fins i tot amb datasets reduïts, com és habitual en entorns mèdics. Tot i ser més lleugera que altres arquitectures de segmentació, manté un alt rendiment, permetent una implementació eficient en sistemes amb recursos computacionals limitats.

### Arquitectura
L'arquitectura d'aquesta xarxa es divideix en 2 parts:

#### Codificador

L'arquitectura del Codificador és prou coneguda per a nosaltres, bàsicament és una CNN seguint les directrius de les xarxes VGG:

De l'article:
>Segueix l'arquitectura típica d'una xarxa convolucional. Consisteix en l'aplicació repetida de dues convolcions amb un kernel de mida 3x3 sense aplicar padding, cada una d'elles seguida d'una ReLU i una operació d'agrupació de max pooling de 2x2 amb stride de 2. A cada pas de reducció de mostreig dupliquem el nombre de canals.

#### Descodificador

El descodificador introdueix una nova operació la _up convolution_ que no és més que una operació que serveix com a oposat de la capa de _max pooling_ del codificador, ens serveix per augmentar la mida del mapa de característiques i vé seguida d'una convolució per canviar el seu nombre de canals a la meitat.

A Pytorch aquesta operació és realitza en una sola capa coneguda com [ConvTranspose2d](https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html) de la documentació oficial tenim aquesta [visualització](https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md).

De l'article:
>Cada passa del descodificador consisteix en un mostreig (_up convolution_) per doblar la mida del mapa de característiques (amplada i alçada)  i que redueix a la meitat el seu nombre de canals, després es realitza una concatenació amb el mapa de característiques corresponent del codificador i dues convolucions amb un kernel de mida 3x3 sense aplicar padding, cada una d'elles seguida d'una capa ReLU. A la capa final s'utilitza una convolució 1x1 per mapejar cada vector de característiques de 64 components al nombre desitjat de classes de sortida

![UNet](img/unet.png "Arquitectura UNet")

En total, la xarxa té 23 capes convolucionals. Per permetre una mosaic perfecte del mapa de segmentació de sortida , és important seleccionar la mida del mosaic d'entrada de manera que totes les operacions de _maxpooling_ de 2x2 s'apliquin a una capa amb una d'amplada i alçada parelles. A causa de les convolucions sense _padding_, la imatge de sortida és més petita que l'entrada.

![mosaic](img/mosaic.png "Arquitectura UNet")

La funció d'activació emprada a la capa de sortida és la _softmax_ ja que en el conjunt de dades hi havia diferents classes de cel·lules a segmentar.






### Exercici

Implementarem el descodificador de la U-net

In [ ]:
import torch.nn as nn
from collections import OrderedDict

class UNet(nn.Module):

    def __init__(self, in_channels=3, out_channels=1, init_features=32):
        super(UNet, self).__init__()

        features = init_features
        
        ## CODER
        self.encoder1 = UNet._block(in_channels, features, name="enc1")
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder2 = UNet._block(features, features * 2, name="enc2")
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder3 = UNet._block(features * 2, features * 4, name="enc3")
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder4 = UNet._block(features * 4, features * 8, name="enc4")
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.bottleneck = UNet._block(features * 8, features * 16, name="bottleneck")
        
        ## DECODER
        
        # TODO: Construeix el teu decoder
        
        self.upconv4 = nn.ConvTranspose2d(¿input?,¿output?, kernel_size=2, stride=2)  # Empra aquesta capa com exemple
        

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        enc4 = self.encoder4(self.pool3(enc3))

        bottleneck = self.bottleneck(self.pool4(enc4))

        # Aquí van les connexions
      
        return ¿?

    
    # Ara ja podem començar a fer coses amb cara i ulls
    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (name + "conv1",
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "relu1", nn.ReLU(inplace=True)),
                    (name + "conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "relu2", nn.ReLU(inplace=True)),
                ]
            )
        )

Ara no farem el procés d'entrenament ho deixarem per la classe de pràctica, però si que es demana:

1. Crear un objecte de la classe UNet.
2. Crear una imatge aleatoria de mida 512x512 i veure que es pot obtenir una sortida de la xarxa.

In [47]:
import numpy as np
import torch
import matplotlib.pyplot as plt

img = np.zeros((1,1,512,512), dtype=np.float32)
t = torch.from_numpy(img)

# TODO: put your code here

img2 = img.detach().numpy()

print(img2.shape)

AttributeError: 'numpy.ndarray' object has no attribute 'detach'

## Mask R-CNN

Mask R-CNN [article](https://arxiv.org/pdf/1703.06870)


Mask R-CNN és una xarxa neuronal dissenyada per resoldre tasques avançades de visió per computador, com ara la segmentació d'objectes, es va presentar l'any 2018. Està basada en una extensió de Faster R-CNN, una arquitectura per a la detecció d'objectes, i afegeix una branca extra que prediu una màscara binària per a cada objecte detectat, permetent identificar no només la seva ubicació sinó també la seva forma precisa. Aquesta xarxa és capaç de processar imatges de manera eficient gràcies a l'ús de regions d'interès (RoIs) que centren els càlculs en àrees específiques de la imatge.

### Història

La història de Mask R-CNN comença amb la línia de recerca en xarxes per a detecció d'objectes i segmentació que va revolucionar el camp de la visió per computador. La seva base es troba l'arquitectura R-CNN (2014), que utilitzava regions proposades per detectar objectes, i les millores successives amb Fast R-CNN (2015) i Faster R-CNN (2015), que van introduir el concepte de xarxes regionals completament convolucionals per millorar la velocitat i eficiència.

### R-CNN

Rich feature hierarchies for accurate object detection and semantic segmentation [article](https://arxiv.org/pdf/1311.2524).

R-CNN va ser una innovació clau perquè va permetre dur a terme la tasca de localitzar objectes dins imatges emprant xarxes convolucionals profundes.

![RCNN](img/RCNN.png "Arquitectura RCNN")


Aquest model es basa en un enfocament de dues etapes. Primer, genera possibles regions d'interès (RoIs) en la imatge utilitzant un mètode extern conegut com Selective Search, que proposa àrees que podrien contenir objectes basant-se en característiques com el color, la textura i els contorns. Cada regió proposada s'ajusta a una mida fixa i es passa a través d'una xarxa convolucional profunda (com AlexNet) per extreure'n un vector de característiques.  Les característiques extretes de la xarxa convolucional es passen a un classificador SVM (Support Vector Machine) separat per assignar una classe a cada regió proposada. Això separa l'extracció de característiques de la classificació, fent el procés menys eficient en comparació amb models posteriors.

Finalment proposa un algorisme de utilitza un regressor lineal entrenat per ajustar els desplaçaments (offsets) de les coordenades de la caixa respecte a la proposta inicial.

R-CNN és computacionalment costosa i lenta perquè processa individualment cadascuna de les regions proposades i perquè s'havien de guardar tots els vectors de característiques en memòria per entrenar les múltiples SVM. Això fa que l'entrenament i la inferència fossin poc pràctics per a aplicacions en temps real.

### Fast R-CNN

Fast R-CNN [article](https://arxiv.org/pdf/1504.08083).

Fast R-CNN (presentada per Ross Girshick el 2015) representa un gran salt endavant en velocitat i eficiència respecte a R-CNN, tot mantenint una alta precisió en la detecció d'objectes.

A diferència de R-CNN, que processa cada regió proposada per separat, Fast R-CNN processa tota la imatge amb una xarxa convolucional compartida, generant un mapa de característiques global. Això elimina la redundància en els càlculs i redueix dràsticament el temps de processament. Les regions proposades es projecten al mapa de característiques que s'extreu de la xarxa i es processen mitjançant una operació anomenada RoI Pooling, que ajusta les dimensions de les regions a una mida fixa. Això permet integrar les propostes de regions dins la xarxa profunda de manera eficient. Fast R-CNN utilitza una única xarxa neuronal per fer dues tasques de manera conjunta mitjançant branques separades a la sortida de la xarxa : 
 - **Classificació**: Assigna una classe a cada RoI.
 - **Regressió de bounding box** (caixes delimitadores): Refina les coordenades de les caixes delimitadores per ajustar-les millor als objectes reals.

Tot el model (classificació i regressió) es pot entrenar conjuntament mitjançant, simplificant el procés en comparació amb R-CNN, que requeria components separats per entrenar SVMs i la regressió. Processant les imatges de manera global Fast R-CNN és molt més ràpid que R-CNN tant durant l'entrenament com en la inferència, tot i que encara depèn d'un mètode extern (com Selective Search) per proposar les regions on es trobaran els objectes.

R-CNN requeria guardar les característiques de cada regió a disc per entrenar SVMs separadament, mentre que Fast R-CNN elimina aquesta necessitat en integrar tots els components en un sol model.

### Faster R-CNN

Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks [article](https://arxiv.org/pdf/1506.01497)

Aquesta artitectura inclou RPN que és una xarxa neuronal integrada en Faster R-CNN que genera automàticament les propostes de regions d'interès. Processa el mapa de característiques de la imatge i retorna una sèrie de caixes delimitadores (bounding boxes) amb puntuacions que indiquen la probabilitat de contenir un objecte.
Això substitueix mètodes externs (lents i computacionalment costosos), fent que Faster R-CNN sigui completament end-to-end (tot el procés desde la imatge incial al resultat està integrat en un sol graf).

Faster R-CNN va marcar un abans i un després en la detecció d'objectes, establint les bases per a moltes altres arquitectures posteriors gràcies al seu disseny modular i eficient.

El següent pas significatiu va ser l'aparició de Mask R-CNN el 2017, desenvolupada per Kaiming He, Georgia Gkioxari, Piotr Dollár i Ross Girshick. Aquesta xarxa va estendre Faster R-CNN afegint una branca específica per a la segmentació semàntica, és a dir, la predicció de màscares pixel a pixel per a cada objecte detectat. A més, va introduir innovacions com el RoIAlign, que solucionava problemes d'alineació durant el procés d'extracció de característiques en regions d'interès, millorant la precisió en les prediccions.


![RCNN](img/comparativa_RCNN.png "Comparativa RCNN")


### Arquitectura
L'arquitectura de Mask R-CNN és una extensió de Faster R-CNN que incorpora una branca addicional per a la segmentació de màscares a nivell de píxel. Aquesta arquitectura combina detecció d'objectes i segmentació d'instàncies en un sol sistema, fent-la molt versàtil per a tasques de visió per computador. Els components principals són els següents:

1. **Xarxa Base** (Feature Extraction Backbone): Utilitza una xarxa convolucional profunda (com ResNet o ResNeXt) com a base per extreure un mapa de característiques de la imatge d'entrada. Sovint inclou un Feature Pyramid Network (FPN) per millorar la representació multiescala, permetent detectar objectes de diferents dimensions amb més precisió.

2. **Xarxa de Proposta de Regions** (Region Proposal Network, RPN): Genera un conjunt de propostes de regions d'interès (RoIs) que podrien contenir objectes. Aquest component utilitza convolucions ancorades per predir bounding boxes i la seva probabilitat de contenir objectes.

3. **RoIAlign** (Regions of Interest Alignment): RoIAlign mapeja amb precisió les propostes de regions al mapa de característiques, millorant la precisió de les prediccions.

4. **Branques de sortida**:

    - Classificació: Determina la classe de cada objecte (o fons).
    - Regressió de bounding boxes: Refina les coordenades de les caixes delimitadores per ajustar-les millor als objectes.
    - Predicció de màscares: Una branca convolucional independent genera una màscara binària a nivell de píxel per a cada objecte detectat. Aquesta màscara té una mida fixa (per exemple, 28x28 píxels) i es pot redimensionar per ajustar-se a la mida de la regió detectada.


El flux general de la xarxa és el següent:

1. Imatge d'entrada: Es processa mitjançant la xarxa base per generar un mapa de característiques.
2. RPN: Es generen propostes de regions d'interès.
3. RoIAlign: Les propostes es mapegen amb precisió al mapa de característiques.
4. Branques: Cada proposta passa per les branques per obtenir la classificació, la caixa delimitadora i la màscara associada.


![MaskRCNN](img/MaskRCNN.png "Arquitectura Mask RCNN")



Mask R-CNN a Pytorch [enllaç](https://pytorch.org/vision/stable/models/mask_rcnn.html)

## Recursos

- [Geeks for Geeks](https://www.geeksforgeeks.org/explain-image-segmentation-techniques-and-applications/#what-is-image-segmentation)
- [Blog Lilian Weng](https://lilianweng.github.io/posts/2017-12-31-object-recognition-part-3/#speed-bottleneck)